## 5-7. 카운트 기반의 문제점과 N-gram을 이용한 보완

### 통계로는 알 수 없는 문맥 정보
지금까지 BOW(Bag of Words) 기반의 방법을 이용해서 다양한 분류 기법을 수행했다. 그러나 BOW에는 치명적인 문제가 있다. 그것은 단어들이 쓰여진 순서에 따른 문맥 정보를 이용할 수 없다는 것이다. 기본적으로 BOW 방식은 단어들의 순서를 무시하고, 단어가 사용된 횟수를 기반으로 문서에 대한 벡터를 만든다. 즉, '나는 학교에 간다'와 '간다 학교에 나는'은 동일한 벡터로 인코딩된다. 이렇게 짧은 문장은 순서가 뒤섞여도 이해가 가능하지만 조금 긴 문장의 경우 대략의 의미는 파악할 수 있지만 정확한 의미를 알기는 어려울 것이다.

이와 같은 문제점을 해결하는 근본적인 방법은 문서를 단어들의 통계적인 값으로 표현하지 않고, 있는 그대로 단어의 시퀀스로 표현해서 처리하는 것이다. 딥러닝 기법은 이와 같은 요구에 의해 사용되기 시작했다고 볼 수 있다. 여기서는 딥러닝 기법 대신 BOW 방식을 그대로 쓰면서도 단어가 쓰여진 순서를 반영할 수 있는 방법인 N-gram에 대해 알아보고자 한다.

### N-gram의 이해
N-gram은 n개의 연속적인 단어들의 나열을 의미한다. 지금까지 토큰화를 할 때, 토큰은 하나의 단어 혹은 의미가 있는 가장 작은 단위로 만들어졌다. 그러나 N-gram에서는 하나의 토큰이 두 개 이상의 단어로 구성될 수 있다. n이 1인 경우 unigram, n이 2이면 bi-gram이라고 부르는데 이 경우 하나의 토큰은 두 개의 단어로 구성된다. n이 3이면 tri-gram이라고 한다.

N-gram의 본래 용도를 이해하려면 먼저 언어 모델에 대해 알아볼 필요가 있다. 언어 모델은 문장 혹은 단어의 시퀀스에 대해 확률을 할당하는 모델을 말하는데, 이때 확률은 말뭉치에 나타난 단어 시퀀스의 빈도와 관련이 있다. 예를 들어, "나는 배가 고파서 밥을 먹었다."와 "나는 배가 고파서 밥을 치웠다." 중에서 앞 문장이 더 자연스럽다는 것을 우리는 알고 있다. 언어 모델은 이 두 문장에 확률을 부여하는데, 우리 상식에 부합하려면 앞 문장에 더 높은 확률을 부여해야 한다. 이에 대한 방법으로 확률을 계산하는 데 참조하는 말뭉치에 앞 문장이 더 많이 나타난다면 이를 기반으로 앞 문장에 더 높은 확률을 부여하면 된다.

그런데 여기에 한 가지 문제가 있다. 말뭉치에서 "나는 배가 고파서 밥을 먹었다."라는 문장은 많이 나왔는데, "나는 배가 고파서 밥을 허겁지겁 먹었다."라는 문장은 나온 적이 없다고 하자. 그러면 언어 모델은 "나는 배가 고파서 밥을 허겁지겁 먹었다."의 확률을 어떻게 계산할까? 우리에게는 자연스럽게 보이지만, 단지 전체 문장이 등장한 적 없다는 이유로 확률이 0이 될 수 있다. N-gram은 이와 같은 문제를 해결하기 위해 만들어졌다. 말뭉치에 앞에서부터 이어지는 전체 문장이 없더라도 "밥을 허겁지겁"과 같은 bi-gram이 말뭉치 어딘가에서 나타났다면 이 문장에 대한 확률이 0이 아니게 할 수 있다.

Bi-gram을 사용하면 적어도 두 단어로 이루어진 시퀀스에 대해서는 파악하게 된다. 따라서 아주 제한적이지만 문맥에 대한 정보를 추가한다고 볼 수 있다. Tri-gram을 사용하면 더 많은 정보를 넣을 수 있을 것이다. 그러나 n을 계속해서 늘려갈 수는 없다. 기본적으로 BOW 기반의 방식은 벡터의 크기가 커서 과적합의 문제가 있다. 여기에 변수를 계속해서 추가하면 문제를 가져올 수밖에 없다. 따라서 많아야 tri-gram 정도까지 쓰는 것이 일반적이다. 따라서 N-gram의 도입에도 불구하고 더 긴 단어 시퀀스로 이루어진 문맥은 여전히 파악할 수 없다는 문제가 아직 남아있다.

### N-gram을 이용한 문서 분류
N-gram의 의미와 한계를 이해했다면 이제 적용을 해본다. 우선은 unigram, bi-gram, tri-gram의 비교를 위해 아래와 같이 Unigram으로 TfidfVectorizer 객체를 새로 생성하고 변환된 TF-IDF 벡터의 크기를 확인한다. N-gram에서 N이 바뀜에 따라 벡터의 크기가 어떻게 바뀌는지 보기 위해 max_features는 사용하지 않는다. 그 외에 토큰화를 위한 정규식을 인수로 주고, NLTK의 불용어사전을 이용한다.

In [1]:
% store -r X_train
% store -r X_test
% store -r y_train
% store -r y_test

In [2]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

cachedStopWords = stopwords.words("english")
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}", # 토큰화를 위한 정규식
                        decode_error="ignore",
                        lowercase=True,
                        stop_words=stopwords.words("english"),
                        max_df=0.5,
                        min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 11483)


주어진 인수 기준으로 토큰화했을 때 남는 단어의 수가 11,483개인 것을 확인했다. 이제 아래와 같이 릿지 회귀분석을 이용해 학습을 하고 성능을 살펴본다. 앞서 설명한 바와 같이 N-gram을 사용하면 변수가 늘어나고 이로 인해 과적합의 우려가 있으므로 릿지 회귀분석을 사용한다.

In [3]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier() # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # 학습
print(f"Train set score {ridge_clf.score(X_train_tfidf, y_train):.3f}")
print(f"Test set score {ridge_clf.score(X_test_tfidf, y_test):.3f}")

Train set score 0.976
Test set score 0.766


이제 bi-gram을 적용해본다. TfidfVectorizer의 ngram_range 매개변수를 사용하면 쉽게 N-gram을 적용할 수 있다. ngram_range 매개변수는 시작 N값과 끝 N값으로 이루어진 튜플을 인수로 받는다. 아래와 같이 (1, 2)로 주면 N이 1과 2가 되므로 unigram과 bi-gram을 모두 사용하게 된다. bi-gram만 사용하고 싶다면 (2, 2)를 인수로 주면 되지만, unigram 없이 학습했을 때 성능이 좋지 않을 것을 짐작할 수 있으므로 원래 변수에 bi-gram을 추가하는 것으로 한다.

In [4]:
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}",
                        decode_error="ignore",
                        lowercase=True,
                        stop_words=stopwords.words("english"),
                        ngram_range=(1, 2), # 바이그램 설정
                        max_df=0.5,
                        min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)            

(2034, 26550)


변환된 결과를 보면 특성의 수가 26,550으로 늘어난 것을 볼 수 있다. max_df와 min_df가 적용되어 늘어난 bi-gram의 숫자가 정확하게 일치하지는 않지만, 전체 특성의 수가 대략 두 배 이상이 된 것을 확인할 수 있다. bi-gram이 어떤 식으로 생성되는지 확인하기 위해 아래와 같이 특성 이름을 출력하고, 릿지 회귀분석으로 학습한 후 결과를 본다.

In [5]:
bigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 1]
print("bi-gram samples:", bigram_features[:10])

ridge_clf.fit(X_train_tfidf, y_train) # 학습
print(f"Train set score: {ridge_clf.score(X_train_tfidf, y_train):.3f}")
print(f"Test set score: {ridge_clf.score(X_test_tfidf, y_test):.3f}")

bi-gram samples: ["'cause can't", "'em better", "'expected errors'", "'karla' next", "'nodis' password", "'official doctrine", "'ok see", "'sci astro'", "'what's moonbase", 'aas american']
Train set score: 0.976
Test set score: 0.773


unigram을 사용했을 때의 정확도인 76.6%에 비해, 77.3%로 약간 성능이 향상됐으므로, bi-gram의 효과가 약간은 있는 것으로 생각할 수 있다. 이제 마지막으로 아래와 같이 tri-gram을 추가하고 학습해서 성능을 본다.

In [6]:
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}",
                        decode_error="ignore",
                        lowercase=True,
                        stop_words=stopwords.words("english"),
                        ngram_range=(1, 3), # 바이그램 설정
                        max_df=0.5,
                        min_df=2)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

trigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 2]
print("tri-gram samples:", trigram_features[:10])

ridge_clf.fit(X_train_tfidf, y_train) # 학습
print(f"Train set score: {ridge_clf.score(X_train_tfidf, y_train):.3f}")
print(f"Test set score: {ridge_clf.score(X_test_tfidf, y_test):.3f}")

(2034, 32943)
tri-gram samples: ["'em better shots", "'expected errors' basically", "'karla' next one", "'nodis' password also", "'official doctrine think", "'ok see warning", "'what's moonbase good", 'aas american astronautical', 'ability means infallible', 'able accept donations']
Train set score: 0.976
Test set score: 0.775


정확도가 77.5%로 bi-gram에 비해서도 약간 성능이 향상됐으나, 별 차이는 없다. 이와 같이 문서 분류에 N-gram을 사용할 경우, 어느 정도 성능의 향상을 기대해볼 수는 있으나 근본적인 해결이라고 보기는 어렵다. 무엇보다 긴 단어 시퀀스로 인한 문맥은 여전히 파악하지 못하고 있다. 그럼에도 불구하고 상황에 따라 N-gram이 필요한 경우가 있으므로 개념과 사용법을 잘 알아 두는 것이 좋다.

In [7]:
% store -z